In [ ]:
import datasets
import pandas as pd
from sklearn.utils import shuffle

# Prompt Collection

In [ ]:
df_oasst = pd.DataFrame(datasets.load_dataset("OpenAssistant/oasst1")['train'])
df_hh_rlhf = pd.DataFrame(datasets.load_dataset('Anthropic/hh-rlhf')['train'])
df_alpaca = pd.DataFrame(datasets.load_dataset('tatsu-lab/alpaca')['train'])
df_synthetic_instruct_gptj_pairwise = pd.DataFrame(datasets.load_dataset('Dahoas/synthetic-instruct-gptj-pairwise')['train'])

New Dataframe with all prompts

In [ ]:
df_prompts = pd.DataFrame(columns=['prompt', 'prompt_origin'])

In [ ]:
df_hh_rlhf['prompt'] = df_hh_rlhf['chosen'].apply(lambda text: text[9:text.find('Assistant:')-2])
df_hh_rlhf['prompt'] = df_hh_rlhf['prompt'].apply(lambda text: text.replace('\n', ' '))
df_hh_rlhf['prompt_origin'] = 'Anthropic/hh_rlhf'
df_prompts = pd.concat([df_prompts, df_hh_rlhf[['prompt', 'prompt_origin']]], ignore_index=True)

In [ ]:
df_synthetic_instruct_gptj_pairwise['prompt_origin'] = 'Dahoas/synthetic_instruct_gptj_pairwise'
df_prompts = pd.concat([df_prompts, df_synthetic_instruct_gptj_pairwise[['prompt', 'prompt_origin']]], ignore_index=True)

In [ ]:
df_oasst = df_oasst[df_oasst['parent_id'].isnull()]
df_oasst = df_oasst[df_oasst['lang'] == 'en']
df_oasst['prompt'] = df_oasst['text']
df_oasst['prompt_origin'] = 'OpenAssistant/oasst1'
df_prompts = pd.concat([df_prompts, df_oasst[['prompt', 'prompt_origin']]], ignore_index=True)

In [ ]:
df_alpaca['instruction'] = df_alpaca['instruction'].apply(lambda text: text.replace('\n', ' '))
df_alpaca['input'] = df_alpaca['input'].apply(lambda text: text.replace('\n', ' '))
df_alpaca['input'] = df_alpaca['input'].apply(lambda text: text if text == '' else ': ' + text)
df_alpaca['instruction'] = df_alpaca.apply(lambda row: row['instruction'][:-1] if row['input'] != '' and row['instruction'].endswith('.') else row['instruction'], axis=1)
df_alpaca['prompt'] = df_alpaca['instruction'] + df_alpaca['input']
df_alpaca['prompt_origin'] = 'tatsu-lab/alpaca'
df_alpaca = df_alpaca[['prompt', 'prompt_origin']]
df_prompts = pd.concat([df_prompts, df_alpaca[['prompt', 'prompt_origin']]], ignore_index=True)

Remove duplicates

In [ ]:
# remove prompt duplicates
df_prompts = df_prompts.drop_duplicates(subset=['prompt'])

Shuffle dataset

In [ ]:
df_prompts = shuffle(df_prompts, random_state=42)
df_prompts

The prompt answering was conducted on the DFKI High-Performance-Computer Cluster